## Colaboradores:
**Christoffer Carlos Rodrigues Evans**

**Gabriel Sousa**

**Heitor Hayashi Ferrairo**

**Henrinque Matheus Silva Souza**

**Larissa Mercês**

In [1]:
# Importações
import json
import os
from IPython.display import clear_output

# Configurações globais
cargos_validos = ['Analista', 'Gerente', 'Traine', 'Senior']  # Lista de cargos permitidos
ARQUIVO = 'funcionarios.json'  # Nome do arquivo para salvar os dados

# Função para carregar dados do arquivo JSON
def carregar_funcionarios():
  if os.path.exists(ARQUIVO):  # Verifica se o arquivo existe
    with open(ARQUIVO, 'r') as f:  # Abre o arquivo para leitura
      return json.load(f)  # Retorna os dados carregados
  else:
    return []  # Retorna lista vazia se o arquivo não existir

# Função para salvar dados no arquivo JSON
def salvar_funcionarios(funcionarios):
  with open(ARQUIVO, 'w') as f:  # Abre o arquivo para escrita
    json.dump(funcionarios, f, indent=2)  # Escreve os dados com formatação

# Exibe um cabeçalho formatado na tela
def cabecalho(titulo):
  print("\033[1m" + "=" * 50)  # Inicia em negrito
  print(titulo.center(50))  # Centraliza
  print("=" * 50)  # Finaliza

def retorna_menu():
  input("Pressione ENTER para continuar...")


#--Funções de validação--

# Valida se o nome contém apenas letras e tem pelo menos 2 caracteres
def validar_nome(nome):
  nome = nome.strip()  # Remove espaços extras
  if len(nome) < 2:  # Verifica tamanho mínimo
    return False
  nome_sem_espaco = nome.replace(" ", "")  # Remove espaços internos
  return nome_sem_espaco.isalpha()  # Verifica se só tem letras

# Valida se o ID é numérico e tem no máximo 6 dígitos
def validar_id(id):
  return id.isnumeric() and len(id) <= 6  # Duas verificações em uma linha

# Verifica se o ID já existe na lista de funcionários
def analisar_id(funcionarios, id_unico):
  for func in funcionarios:  # Percorre todos os funcionários
    if func['id'] == id_unico:  # Se encontrar o ID
      return False  # ID já existe

  return True  # ID é único

# Encontra a posição de um funcionário na lista pelo ID
def encontrar_funcionario_por_id(funcionarios, id_busca):
  for i, func in enumerate(funcionarios):  # Percorre com índice
    if func['id'] == id_busca:  # Se encontrar o ID
      return i  # Retorna a posição

  return -1  # Retorna -1 se não encontrar


#--Operações CRUD--

# Lista todos os funcionários em ordem alfabética
def listar_funcionarios(funcionarios):
  cabecalho("LISTAGEM DE FUNCIONÁRIOS")

  if not funcionarios:  # Lista vazia
    print("Nenhum funcionário cadastrado.\n")
  else:
    # Ordena os funcionários pelo nome (case-insensitive)
    funcionarios_ordenados = sorted(funcionarios, key=lambda x: x['nome'].lower())

    # Exibe cada funcionário formatado
    for i, func in enumerate(funcionarios_ordenados, 1):  # Começa em 1
      print(f"{i}. Nome : {func['nome']}")
      print(f"   Cargo: {func['cargo']}")
      print(f"   ID   : {func['id']}")
      print("-" * 50)  # Linha separadora

  retorna_menu()

# Cadastra um novo funcionário com dados validados
def cadastrar_funcionario(funcionarios):
  cabecalho("CADASTRAR FUNCIONÁRIO")

  # Validação do nome
  while True:
    nome = input('Nome: ').strip()
    if validar_nome(nome):
      break
    print('Nome inválido. Deve conter apenas letras e espaço para nome/sobrenome.')

  # Validação do cargo
  print("Cargos válidos: " + ", ".join(cargos_validos))
  while True:
    cargo = input('Cargo: ').strip().capitalize()  # Padroniza capitalização
    if cargo in cargos_validos:
      break

    print('Cargo inválido. Escolha entre: ' + ", ".join(cargos_validos))

  # Validação do ID
  while True:
    id_func = input('ID: ').strip()
    if not validar_id(id_func):
      print('ID inválido. Deve conter apenas números (máx. 6 dígitos).')
      continue  # Volta ao início do loop

    if not analisar_id(funcionarios, id_func):
      print('ID já cadastrado. Escolha outro.')
      continue
    break  # ID válido, sai do loop

  # Cria e adiciona o novo funcionário
  novo_func = {'nome': nome, 'cargo': cargo, 'id': id_func}
  funcionarios.append(novo_func)
  salvar_funcionarios(funcionarios)  # Salva no arquivo

  print('Funcionário cadastrado com sucesso!')
  retorna_menu()

# Edita os dados de um funcionário existente
def editar_funcionario(funcionarios):
  cabecalho("EDITAR FUNCIONÁRIO")

  # Lista vazia
  if not funcionarios:
    print("Nenhum funcionário cadastrado.")
    retorna_menu()
    return

  # Busca pelo ID
  id_editar = input("Digite o ID: ").strip()
  pos = encontrar_funcionario_por_id(funcionarios, id_editar)
  if pos == -1:  # Não encontrado
    print("Funcionário não encontrado.")
    retorna_menu()
    return

  func = funcionarios[pos]  # Funcionário encontrado

  # Edição do nome (opcional)
  print(f"Nome atual: {func['nome']}")
  novo_nome = input("Novo nome (ENTER mantém): ").strip()
  if novo_nome and validar_nome(novo_nome):  # Se digitou e é válido
    func['nome'] = novo_nome

  # Edição do cargo (opcional)
  print("Cargos válidos: " + ", ".join(cargos_validos))
  novo_cargo = input("Novo cargo (ENTER mantém): ").strip().capitalize()
  if novo_cargo and novo_cargo in cargos_validos:  # Se digitou e é válido
    func['cargo'] = novo_cargo

  # Atualiza e salva
  funcionarios[pos] = func
  salvar_funcionarios(funcionarios)
  print("Funcionário atualizado com sucesso!")
  retorna_menu()

# Remove um funcionário da lista após confirmação
def remover_funcionario(funcionarios):
  cabecalho("REMOVER FUNCIONÁRIO")
  if not funcionarios:  # Lista vazia
    print("Nenhum funcionário cadastrado.")
    retorna_menu()
    return

  # Busca pelo ID
  id_remover = input("Digite o ID: ").strip()
  pos = encontrar_funcionario_por_id(funcionarios, id_remover)
  if pos == -1:  # Não encontrado
    print("Funcionário não encontrado.")
  else:
    # Confirmação antes de remover
    confirmar = input(f"Remover {funcionarios[pos]['nome']}? (s/n): ").strip().lower()

    if confirmar == 's':  # Usuário confirmou
      funcionarios.pop(pos)  # Remove o funcionário
      salvar_funcionarios(funcionarios)  # Salva alterações
      print("Funcionário removido com sucesso!")

    else:  # Usuário cancelou
      print("Exclusão cancelada.")

  retorna_menu()

# Busca e exibe um funcionário pelo ID
def identificar_funcionario(funcionarios):
  cabecalho("BUSCAR FUNCIONÁRIO")

  if not funcionarios:  # Lista vazia
    print("Nenhum funcionário cadastrado.")
    retorna_menu()
    return

  # Busca pelo ID
  id_busca = input("Digite o ID: ").strip()
  pos = encontrar_funcionario_por_id(funcionarios, id_busca)

  if pos == -1:  # Não encontrado
    print("Funcionário não encontrado.")

  else:  # Encontrado - exibe dados
    func = funcionarios[pos]
    print(f"Nome: {func['nome']}")
    print(f"Cargo: {func['cargo']}")
    print(f"ID: {func['id']}")

  retorna_menu()

# Busca funcinários por cargo
def identificar_por_cargo(funcionarios):
  cabecalho("BUSCAR POR CARGO")

  # Verifica se a lista não está vazia
  if not funcionarios:
    print('Nenhum funcionario cadastrado')
    retorna_menu()
    return

  # Pede o termo e deixa a primeira letra maiuscula
  termo = input('Digite o cargo para identificar o funcionario: ').strip().capitalize()

  # Lista de funcionários encontrados
  funcionarios_encontrados = []

  # Checa todos os funcionários que tem o mesmo cargo e adiciona eles a lista
  for f in funcionarios:
    cargo = f['cargo']
    if termo in cargo:
      funcionarios_encontrados.append(f)

  # Se tiver funcionários na lista exibe
  if len(funcionarios_encontrados) > 0:

    for f in funcionarios_encontrados:
      print(f"Nome : {f['nome']}")
      print(f"Cargo: {f['cargo']}")
      print(f"ID: {f['id']}")
      print("-" * 50)

  else:
    print('Nenhum funcionario encontrado.')

  retorna_menu()


#--Sistema principal--

# Exibe o menu principal e gerencia as operações
def menu():
  funcionarios = carregar_funcionarios()  # Carrega dados ao iniciar

  while True:  # Loop principal
    clear_output()  # Limpa a tela
    cabecalho("📁 SISTEMA DE FUNCIONÁRIOS")

    print("".center(15) + "1️⃣ Listar Funcionários")
    print("".center(15) + "2️⃣ Cadastrar Funcionário")
    print("".center(15) + "3️⃣ Editar Funcionário")
    print("".center(15) + "4️⃣ Remover Funcionário")
    print("".center(15) + "5️⃣ Buscar Funcionário")
    print("".center(15) + "6️⃣ Busca por Cargo")
    print("".center(15) + "7️⃣ 🚪 Sair")

    opcao = input("\n👉 Escolha: ").strip()

    # Processa a opção escolhida
    if opcao == '1':
      listar_funcionarios(funcionarios)
    elif opcao == '2':
      cadastrar_funcionario(funcionarios)
    elif opcao == '3':
      editar_funcionario(funcionarios)
    elif opcao == '4':
      remover_funcionario(funcionarios)
    elif opcao == '5':
      identificar_funcionario(funcionarios)
    elif opcao == '6':
      identificar_por_cargo(funcionarios)
    elif opcao == '7':
      print("Encerrando programa...")
      break

    else:  # Opção inválida
      print("Opção inválida.")
      retorna_menu()

# Entrada do programa
try:
  menu()
except KeyboardInterrupt:
  print("Programa encerrado manualmente")
except Exception:
  print(f"Ocorreu um erro inesperado")
finally:
  print("Programa encerrado.")

            📁 SISTEMA DE FUNCIONÁRIOS             
               1️⃣ Listar Funcionários
               2️⃣ Cadastrar Funcionário
               3️⃣ Editar Funcionário
               4️⃣ Remover Funcionário
               5️⃣ Buscar Funcionário
               6️⃣ Busca por Cargo
               7️⃣ 🚪 Sair

👉 Escolha: 7
Encerrando programa...
Programa encerrado.
